In [1]:
from RiotAPI import *
import pickle
import time

# 사전준비   
랭크별 summoner name의 list   
https://developer.riotgames.com/api-methods/#league-v4/GET_getLeagueEntries
- tiers = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
- divisions = ['I', 'II', 'III', 'IV']
- queue = ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']  

랭크별 account의 list: { 랭크이름: account_id의 list } 

In [ ]:
tier = 'BRONZE'
divisions = ['I', 'II', 'III', 'IV']
queue = 'RANKED_SOLO_5x5' # 솔랭을 기준으로

summoners = []

for div in divisions:
    summoners.extend(get_entry(queue, tier, div))

In [ ]:
# get a list of account id for a list of summoner id
accounts = []
error = []

for index, smr in enumerate(summoners):
    try:
        accounts.append(get_account_id_by_summonerId(smr))
        print("Summoner No.{} Collected".format(str(index)))
    except:
        pass

In [ ]:
accounts[:5]

In [ ]:
# save as pickle
fn = "pickle/tmp_bronze_account.pickle"
with open(fn, 'wb') as f:
    pickle.dump(accounts, f)

fn = "pickle/tmp_bronze_summoner.pickle"
with open(fn, 'wb') as f:
    pickle.dump(summoners, f)

error = ['베어마흐트', 'GuiItyCronw', 'haowugujide', '족장병만']

In [2]:
# 820
with open('pickle/bronze_account.pickle', 'rb') as f:
    full_account = pickle.load(f)

# 816
with open('pickle/tmp_bronze_account.pickle', 'rb') as f:
    broken_account = pickle.load(f)

In [3]:
# 820개 account id 중에서 816개 tmp account에 없는 것들 matchreference 추가적으로 뽑기
changed = [usr for usr in full_account if usr not in broken_account]

In [4]:
changed # 왜 여섯개가 나오지

['nY0bou1YncM-r9dEy2t2jyCY7OGP6kr0NnoUKTmGwzDz',
 'iPoSU8Bwma_Hph8oZR2j4A_KgYDuuet78u9JkgtI5NMs',
 'URJiY4FcLFm8eQtp4kZ6WffCJ2QlwDJ3f10KTXyC4Osw',
 'zPq2yhj0RmFZd6aYw_ZrNL0dTJVA2IRXltvisMdsBibg',
 'D-ZW3os8KhmSEuYxxGYq_8qdsTnBADe8Dz6Rhu6kDK5Y',
 'cIbXM8NIQnfUHLaGbRIS_zuI1hLY_uWY_IeGPaFPkq2e']

# account_id 별 MatchReference
https://developer.riotgames.com/api-methods/#match-v4/GET_getMatchlist   
: { account_id : MatchReference의 list }

In [ ]:
hst_error =[]
match_history = {}

for index, acc in enumerate(accounts):
    try:
        match_history[acc] = get_match_history(acc)
        print("Match History for Account No.{} Collected".format(str(index)))
    except:
        hst_error.append(acc)

In [ ]:
# save data
fn = "pickle/tmp_bronze_match_history.pickle"
with open(fn, 'wb') as f:
    pickle.dump(match_history, f)

fn = "pickle/tmp_bronze_match_history_error.pickle"
with open(fn, 'wb') as f:
    pickle.dump(hst_error, f)

In [ ]:
print(len(match_history))
match_history[accounts[0]][:5]

In [ ]:
print(len(hst_error))
hst_error

# match_id 의 MatchDto  
https://developer.riotgames.com/api-methods/#match-v4/GET_getMatch   
: {match_id: Match}

In [16]:
with open('pickle/tmp_bronze_match_history.pickle', 'rb') as f:
    match_history = pickle.load(f)

# list up unique game id
games = [match['gameId'] for key, matchList in match_history.items() for match in matchList]
unq_games = list(set(games))

print(len(unq_games))
print(unq_games[:10])

698490
[2856321024, 3596615682, 3575644165, 3227269439, 3125805063, 3253731338, 3084910603, 3017801738, 2683305997, 2625634318]


In [30]:
fn = "pickle/unq_games.pickle"
with open(fn, 'wb') as f:
    pickle.dump(unq_games, f)

In [ ]:
for i in range(0, 6984):
    
    # save as file for every 100 game detail
    rng = (i*100, (i+1)*100)

    match_detail = {} # dictionary to save game detail
    detail_error = [] # list of broken match id

    # starting requests
    start = time.time() 
    
    for index, match in unq_games[rng[0]:rng[1]]:
        try:
            match_detail[match] = get_match_detail(match)
        except:
            print("Match No.{} Broken".format(str(index)))
            detail_error.append(match)
    
    # requests completed
    end = time.time() 
    delta = (end - start)/60
    print("======= {0}th 100 Matches Completed: took {1} minutes =====".format(str(i), str(delta))
    
    # save as pickle
    fn = "pickle/detail/match_detail"+str(rng[0])+"_"+str(rng[1])+".pickle"
    with open(fn, 'wb') as f:
        pickle.dump(match_detail, f)

    fn = "pickle/detail/detail_error"+str(rng[0])+"_"+str(rng[1])+".pickle"
    with open(fn, 'wb') as f:
        pickle.dump(detail_error, f)